# 在JupyterLab中运行R

1. 在R中安装必要的包，安装方法：
    ```r
    install.packages(c('repr', 'IRdisplay', 'evaluate', 'crayon', 'pbdZMQ', 'devtools', 'uuid', 'digest', 'IRkernel))
    ```
1. 安装R内核
    - 当前用户
    ```r
    IRkernel::installspec()
    ```
    - 系统所有用户
    ```r
    IRkernel::installspec(user=FALSE)
    ```
1. 启动数据库`localhost/sqlzoo`（数据需提前导入）。
1. 通过`DBI`连接sqlzoo数据库。

本案例使用PostgreSQL，也可以使用MySQL/MariaDB。

In [1]:
library(tidyverse)
library(DBI)
library(getPass)
drv <- switch(Sys.info()['sysname'],
             Windows="PostgreSQL Unicode(x64)",
             Darwin="/usr/local/lib/psqlodbcw.so",
             Linux="PostgreSQL")
con <- dbConnect(
  odbc::odbc(),
  driver = drv,
  Server = "localhost",
  Database = "sqlzoo",
  UID = "postgres",
  PWD = getPass("Password?"),
  Port = 5432
)

─ Attaching packages ──────────────────── tidyverse 1.3.0 ─

✔ ggplot2 3.3.0     ✔ purrr   0.3.4
✔ tibble  3.0.1     ✔ dplyr   0.8.5
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

─ Conflicts ───────────────────── tidyverse_conflicts() ─
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



Password? ····


连接上以后还可以做一些设置，比如将长度显示行数的上限为10：

In [2]:
options(repr.matrix.max.rows=10)

## 示例

接下来就可以用`dplyr`操作数据库了。

In [3]:
library(tidyverse)
world <- dbReadTable(con, "world")
world

name,continent,area,population,gdp,capital,tld,flag
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
Afghanistan,Asia,652230,25500100,2.03640e+10,Kabul,.af,//upload.wikimedia.org/wikipedia/commons/9/9a/Flag_of_Afghanistan.svg
Albania,Europe,28748,2821977,1.20440e+10,Tirana,.al,//upload.wikimedia.org/wikipedia/commons/3/36/Flag_of_Albania.svg
Algeria,Africa,2381741,38700000,2.07021e+11,Algiers,.dz,//upload.wikimedia.org/wikipedia/commons/7/77/Flag_of_Algeria.svg
Andorra,Europe,468,76098,3.22200e+09,Andorra la Vella,.ad,//upload.wikimedia.org/wikipedia/commons/1/19/Flag_of_Andorra.svg
Angola,Africa,1246700,19183590,1.16308e+11,Luanda,.ao,//upload.wikimedia.org/wikipedia/commons/9/9d/Flag_of_Angola.svg
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Venezuela,South America,916445,28946101,3.82424e+11,Caracas,.ve,//upload.wikimedia.org/wikipedia/commons/0/06/Flag_of_Venezuela.svg
Vietnam,Asia,331212,89708900,1.55820e+11,Hanoi,.vn,//upload.wikimedia.org/wikipedia/commons/2/21/Flag_of_Vietnam.svg
Yemen,Asia,527968,25235000,3.28310e+10,Sana‘a,.ye,//upload.wikimedia.org/wikipedia/commons/8/89/Flag_of_Yemen.svg


`world`表一共195行，但由于全局设置了`options(repr.matrix.max.rows)`，默认只显示了10行（前5+后5）。